# Практическая работа №2. Применение классических методов машинного обучения в среде Apache Spark

### Необходимые зависимости (запустить)

In [ ]:
!git clone --recursive https://github.com/tester170/Other.git

Cloning into 'Other'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 56 (delta 13), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (56/56), 32.77 MiB | 8.12 MiB/s, done.
Resolving deltas: 100% (13/13), done.
Updating files: 100% (26/26), done.


In [ ]:
!ls Other/

archive.zip  beauty.csv  columns.csv  email.csv  images		 README.md	xl
bank.csv     births.csv  data.zip     excel.py	 passengers.csv  responses.csv	сollege_data.csv


In [ ]:
!unzip "/content/Other/data.zip" -d "/content/"

Archive:  /content/Other/data.zip
   creating: /content/data/
  inflating: /content/__MACOSX/._data  
  inflating: /content/data/new_customers.csv  
  inflating: /content/__MACOSX/data/._new_customers.csv  
   creating: /content/data/ml-1m/
  inflating: /content/__MACOSX/data/._ml-1m  
  inflating: /content/data/Meal_Info.csv  
  inflating: /content/__MACOSX/data/._Meal_Info.csv  
  inflating: /content/data/fakefriends.csv  
  inflating: /content/__MACOSX/data/._fakefriends.csv  
  inflating: /content/data/sample_linear_regression_data.txt  
  inflating: /content/__MACOSX/data/._sample_linear_regression_data.txt  
  inflating: /content/data/College.csv  
  inflating: /content/__MACOSX/data/._College.csv  
  inflating: /content/data/fake_customers.csv  
  inflating: /content/__MACOSX/data/._fake_customers.csv  
  inflating: /content/data/movielens_ratings.csv  
  inflating: /content/__MACOSX/data/._movielens_ratings.csv  
  inflating: /content/data/Book.txt  
  inflating: /content/__MAC

In [ ]:
!pip install pyspark
!pip install findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('Practice2').getOrCreate()

## Задание №1. Линейная регрессия (Linear Regression)

### Построение прогностической модели для судов

[Hyundai Heavy Industries](http://www.hyundai.eu/en) - одна из ведущих судостроительных компаний в мире, специализирующаяся на строительстве круизных лайнеров. В настоящее время они работают над созданием новых кораблей для различных клиентов и стремятся к точному прогнозированию количества необходимых членов экипажа для каждого корабля.

Данные, с которыми мы работаем, представляют собой измерения размеров, вместимости, экипажа и возраста 158 круизных судов. Эти данные включают следующие столбцы:

- Ship_name
- Cruise_line
- Age (по состоянию на 2013 год)
- Tonnage (1000 тонн)
- Passengers (100 человек)
- Length (100 футов)
- Cabins (100 шт.)
- Passenger_density
- Crew (100s)

Эти данные хранятся в файле CSV под названием "cruise_ship_info.csv".

##### **Ваша задача - создать регрессионную модель, которая поможет предсказать, сколько членов экипажа потребуется для будущих кораблей.**

In [ ]:
df = spark.read.csv('data/cruise_ship_info.csv',inferSchema=True,header=True)

In [ ]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [ ]:
df.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [ ]:
df.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       NULL|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|     NULL|       NULL| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [ ]:
# Прежде чем Spark сможет принять данные, нам нужно сделать несколько преобразований
# Данные должны быть в виде двух столбцов

# ("label", "features")

# Импортируем VectorAssembler и Vectors

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [ ]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [ ]:
# Создаем экземпляр класса VectorAssembler
assembler = VectorAssembler(
    # Указываем входные колонки, которые будут использоваться для создания вектора признаков
    inputCols=["Age", "Tonnage", "passengers", "length", "cabins", "passenger_density"],
    # Указываем имя выходной колонки, в которой будет храниться вектор признаков
    outputCol="features")

In [ ]:
# Применяем assembler к данным для создания вектора признаков
output = assembler.transform(df)

In [ ]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,30.276999999...|
|[26.0,47.262,14.8...|
|[11.0,110.0,29.74...|
|[17.0,101.353,26....|
|[22.0,70.367,20.5...|
|[15.0,70.367,20.5...|
|[23.0,70.367,20.5...|
|[19.0,70.367,20.5...|
|[6.0,110.23899999...|
|[10.0,110.0,29.74...|
|[28.0,46.052,14.5...|
|[18.0,70.367,20.5...|
|[17.0,70.367,20.5...|
|[11.0,86.0,21.24,...|
|[8.0,110.0,29.74,...|
|[9.0,88.5,21.24,9...|
|[15.0,70.367,20.5...|
|[12.0,88.5,21.24,...|
|[20.0,70.367,20.5...|
+--------------------+
only showing top 20 rows



In [ ]:
output.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|[6.0,30.276999999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|[6.0,30.276999999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|[26.0,47.262,14.8...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|[11.0,110.0,29.74...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|[17.0,101.353,26....|
|    Ecstasy|   Carnival| 22|            70.367|     20.

In [ ]:
# Выбираем колонки 'features' и 'Yearly Amount Spent' из DataFrame 'output'

final_data = output.select("features",'crew')

In [ ]:
# Разделяем данные на обучающий и тестовый наборы
# 70% данных идут в обучающий набор, 30% - в тестовый

train_data, test_data = final_data.randomSplit([0.7,0.3])

In [ ]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               106|
|   mean| 7.754056603773596|
| stddev|3.5684100176280684|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [ ]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                52|
|   mean| 7.875961538461537|
| stddev|3.3999493996323435|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+



In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
# Создаём объект модели линейной регрессии

lr = LinearRegression(labelCol='crew')

In [ ]:
# Обучим модель

lrModel = lr.fit(train_data,)

In [ ]:
# Выводим коэффициенты и пересечение с осью Y (intercept) обученной модели

print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [-0.02047354800025383,-0.00047048423927912355,-0.14142220493650182,0.41212014150599946,0.894338384558032,-0.0015125788263620297] Intercept: -0.39932616379135355


In [ ]:
test_results = lrModel.evaluate(test_data)

In [ ]:
# Интересные результаты....
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| 0.44855865738958123|
| -1.4500512657716786|
|-0.44085714190043923|
|-0.30309334630171136|
|   -0.80620496902708|
| -0.7413598443872065|
| 0.32800063534144464|
| -0.4915164983388358|
| -0.5770392961219386|
| -0.7250074978020127|
|   0.985737408773355|
|   0.985737408773355|
|  1.7961501000506783|
|-0.33847378274229456|
|   0.444036115320273|
| 0.30157830638485983|
|-0.47297220051017064|
| -0.7004127483866984|
|  2.1909693745965626|
|  0.9019623085672812|
+--------------------+
only showing top 20 rows



In [ ]:
# Создаем немаркированный набор данных, выбирая только признаки из тестовых данных

unlabeled_data = test_data.select('features')

In [ ]:
# Применяем модель к немаркированным данным для получения прогнозов

predictions = lrModel.transform(unlabeled_data)

In [ ]:
# Выводим прогнозы

predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,115.0,35.74,...|11.751441342610418|
|[5.0,160.0,36.34,...|15.050051265771678|
|[6.0,112.0,38.0,9...| 11.34085714190044|
|[6.0,158.0,43.7,1...|13.903093346301711|
|[7.0,116.0,31.0,9...| 12.80620496902708|
|[8.0,110.0,29.74,...|12.341359844387206|
|[9.0,81.0,21.44,9...| 9.671999364658555|
|[9.0,85.0,19.68,9...| 9.181516498338835|
|[9.0,105.0,27.2,8...|11.257039296121938|
|[9.0,110.0,29.74,...|12.325007497802012|
|[9.0,113.0,26.74,...|11.394262591226646|
|[9.0,113.0,26.74,...|11.394262591226646|
|[10.0,46.0,7.0,6....|2.6738498999493214|
|[10.0,58.825,15.6...| 7.338473782742295|
|[10.0,68.0,10.8,7...| 5.915963884679727|
|[10.0,77.0,20.16,...|  8.69842169361514|
|[10.0,81.76899999...|  8.89297220051017|
|[10.0,110.0,29.74...|12.300412748386698|
|[10.0,151.4,26.2,...|10.339030625403437|
|[11.0,90.0,22.4,9...|10.098037691432719|
+--------------------+------------

In [ ]:
# Выводим корень из среднеквадратической ошибки (RMSE)

print("RMSE: {}".format(test_results.rootMeanSquaredError))

# Выводим среднеквадратическую ошибку (MSE)

print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 0.7528938869636457
MSE: 0.5668492050272269


## Задание №2. Логистическая регрессия (Logistic Regression)

## Разработка модели бинарного классификатора для определения оттока клиентов

- Маркетинговое агентство имеет множество клиентов, которые используют их услуги для создания рекламы на веб-сайтах клиентов. Они заметили, что у них довольно большой отток клиентов. Необходимо создать модель машинного обучения, которая поможет предсказать, какие клиенты уйдут (перестанут покупать их услуги), чтобы они могли правильно назначить менеджера по работе с клиентами, которые наиболее вероятно уйдут.

- **Разработайте алгоритм классификации, который поможет определить, уйдет клиент или нет.** Затем компания может проверить это на входящих данных для будущих клиентов, чтобы предсказать, какие клиенты уйдут и назначить им менеджера по работе с клиентами.

Данные сохранены как customer_churn.csv. Вот поля и их определения:

    Name : Имя последнего контакта в компании
    Age: Возраст клиента
    Total_Purchase: Всего куплено рекламы
    Account_Manager: Бинарный 0=Нет менеджера, 1= Назначен менеджер по работе с клиентами
    Years: Общее количество лет в качестве клиента
    Num_sites: Количество веб-сайтов, которые используют услугу.
    Onboard_date: Дата, когда последний контакт был привлечен
    Location: Адрес головного офиса клиента
    Company: Название клиентской компании
    
После того как вы создали модель и оценили ее, проверьте модель на новых данных, которые сохранены под именем new_customers.csv. Необходимо узнать, какие клиенты наиболее вероятно уйдут (у них еще нет метки).

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Logistic Regression').getOrCreate()
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
data = spark.read.csv('data/customer_churn.csv',inferSchema=True,
                     header=True)

In [ ]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [ ]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         NULL|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                NULL|                NULL|0.16666666666666666|
| stddev|         NULL|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [ ]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [ ]:
data = data.select(['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Company',
 'Churn'])

In [ ]:
data = data.na.drop()

In [ ]:
# Импортируем необходимые классы из модуля pyspark.ml.feature
from pyspark.ml.feature import (
    VectorAssembler,  # Класс для сбора нескольких столбцов в один векторный столбец
    VectorIndexer,    # Класс для автоматического индексирования категориальных признаков
    OneHotEncoder,    # Класс для преобразования категориальных признаков в бинарные векторы
    StringIndexer     # Класс для индексирования строковых столбцов
)

In [ ]:
# Создаем объект StringIndexer. Этот объект преобразует строковые значения в индексы.
# Входной столбец - 'Sex', выходной столбец - 'SexIndex'.
company_indexer = StringIndexer(inputCol='Company',outputCol='CompanyIndex', handleInvalid="keep")

# Создаем объект OneHotEncoder. Этот объект преобразует категориальные признаки, закодированные как индексы, в векторы One-Hot.
# Входной столбец - 'SexIndex', выходной столбец - 'SexVec'.
company_encoder = OneHotEncoder(inputCol='CompanyIndex',outputCol='CompanyVec')

In [ ]:
# Создаем объект VectorAssembler. Этот объект объединяет заданный список столбцов в один векторный столбец.
# Входные столбцы - 'Pclass', 'SexVec', 'Age', 'SibSp', 'Parch', 'Fare', 'EmbarkVec'.
# Выходной столбец - 'features'.
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'CompanyVec'],outputCol='features')

In [ ]:
# Создаем объект LogisticRegression. Этот объект представляет собой модель логистической регрессии.
# Столбец признаков (featuresCol) - 'features', столбец меток (labelCol) - 'Survived'.
log_reg = LogisticRegression(featuresCol='features',labelCol='Churn')

In [ ]:
# Создаем объект Pipeline. Этот объект представляет собой последовательность этапов обработки данных.
# Этапы включают: gender_indexer, embark_indexer, gender_encoder, embark_encoder, assembler, log_reg_titanic.
pipeline = Pipeline(stages=[
    company_indexer,  # Преобразование строковых значений в 'Sex' в индексы
    company_encoder,  # Преобразование строковых значений в 'Embarked' в индексы
    assembler,       # Объединение нескольких признаков в один векторный столбец 'features'
    log_reg  # Применение модели логистической регрессии
])


In [ ]:
# Метод randomSplit разделяет исходные данные на два набора данных в заданной пропорции.
# В данном случае, мы разделяем данные на обучающий набор (70% данных) и тестовый набор (30% данных).

train, test = data.randomSplit([0.7,.3])

In [ ]:
# Метод fit обучает модель на обучающих данных.
# В данном случае, мы обучаем наш конвейер (pipeline) на обучающем наборе данных (train_titanic_data).

fit_model = pipeline.fit(train)

In [ ]:
# Метод transform применяет обученную модель к новым данным.
# В данном случае, мы применяем нашу обученную модель (fit_model) к тестовому набору данных (test_titanic_data).

results = fit_model.transform(test)

In [ ]:
# Создаем объект BinaryClassificationEvaluator. Этот объект представляет собой оценщик для бинарных классификационных моделей.
# Столбец прогнозов (rawPredictionCol) - 'prediction', столбец меток (labelCol) - 'Survived'.
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                        labelCol='Churn')


In [ ]:
results.select('Churn','prediction').show()

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



In [ ]:
# Метод evaluate применяет оценщик к результатам модели.
# В данном случае, мы применяем наш оценщик (my_eval) к результатам модели (results).
# Результатом является значение метрики AUC (Area Under the ROC Curve).

AUC = my_eval.evaluate(results)

In [ ]:
AUC

0.6136448598130841

## Задание №3. Модели основанные на решающих деревьях (Решающие деревья, Случайный лес, Градиентный бустинг)

- Для компании-произвозителя собачьего корма, необходимо попытаться предсказать, почему некоторые партии их корма портятся гораздо быстрее, чем предполагалось.

- К сожалению, эта компания по производству собачьего корма не обновила свое оборудование до последних моделей, что означает, что количество пяти консервантов, которые они используют, может значительно варьироваться. Но какой из консервантов оказывает наибольшее влияние?

- Сначала компания по производству собачьего корма готовит партию консерванта, которая содержит 4 разных консерванта (A, B, C, D), а затем добавляет "наполнитель". Ученые полагают, что один из консервантов A, B, C или D вызывает проблему и необходимо выяснить, какой именно.

Используйте методы машинного обучения, основанные на решающих деревьях, чтобы выяснить, какой признак имеет наибольшее значение, а затем выясните, какой химический элемент вызывает раннюю порчу.

* Pres_A : Процент консерванта A в смеси
* Pres_B : Процент консерванта B в смеси
* Pres_C : Процент консерванта C в смеси
* Pres_D : Процент консерванта D в смеси
* Spoiled: Метка, указывающая, испортилась ли партия собачьего корма.

___

**Тщательно подумайте о том, что на самом деле требуется решить в этой задаче!**
____

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rf').getOrCreate()

In [ ]:
data = spark.read.csv('data/dog_food.csv',inferSchema=True,header=True)

In [ ]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [ ]:
data.head()

Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0)

In [ ]:
data.describe().show()

+-------+------------------+------------------+------------------+------------------+-------------------+
|summary|                 A|                 B|                 C|                 D|            Spoiled|
+-------+------------------+------------------+------------------+------------------+-------------------+
|  count|               490|               490|               490|               490|                490|
|   mean|  5.53469387755102| 5.504081632653061| 9.126530612244897| 5.579591836734694| 0.2857142857142857|
| stddev|2.9515204234399057|2.8537966089662063|2.0555451971054275|2.8548369309982857|0.45221563164613465|
|    min|                 1|                 1|               5.0|                 1|                0.0|
|    max|                10|                10|              14.0|                10|                1.0|
+-------+------------------+------------------+------------------+------------------+-------------------+



In [ ]:
(train, test) = data.randomSplit([0.7, 0.3])

In [ ]:
train.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [ ]:
# Создаем объект VectorAssembler. Этот объект объединяет заданный список столбцов в один векторный столбец.
# Входные столбцы - 'Pclass', 'SexVec', 'Age', 'SibSp', 'Parch', 'Fare', 'EmbarkVec'.
# Выходной столбец - 'features'.
assembler = VectorAssembler(inputCols=['A',
 'B',
 'C',
 'D',
 ],outputCol='features')
train = assembler.transform(train)
test = assembler.transform(test)

In [ ]:
# Создание экземпляра классификатора случайного леса

rf = RandomForestClassifier(labelCol="Spoiled", featuresCol="features", numTrees=20)

In [ ]:
# Обучение модели на основе обучающих данных.

model = rf.fit(train)

In [ ]:
# Применение обученной модели для прогнозирования на тестовых данных.

predictions = model.transform(test)

In [ ]:
predictions.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [ ]:
predictions.select("prediction", "Spoiled", "features").show(5)

+----------+-------+-------------------+
|prediction|Spoiled|           features|
+----------+-------+-------------------+
|       1.0|    1.0| [1.0,1.0,13.0,3.0]|
|       0.0|    0.0|  [1.0,2.0,9.0,1.0]|
|       0.0|    0.0|  [1.0,3.0,8.0,3.0]|
|       0.0|    0.0|  [1.0,4.0,8.0,7.0]|
|       1.0|    1.0|[1.0,4.0,13.0,10.0]|
+----------+-------+-------------------+
only showing top 5 rows



Оценка точности:

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="Spoiled", predictionCol="prediction", metricName="accuracy")

In [ ]:
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.0275862


In [ ]:
# Получение важности признаков модели.

model.featureImportances

SparseVector(4, {0: 0.0186, 1: 0.0196, 2: 0.9431, 3: 0.0188})

Исходя из важности признаков, можно сделать вывод, что на порчу корма влияет консервант С.

# Задание №4. Кластеризация

- Крупная технологическая компания подверглась хакерской атаке. К счастью, их судебные инженеры смогли собрать ценные данные о взломах, включая информацию о продолжительности сессии, местоположении, скорости печати и т.д. Судебный инженер рассказывает, что удалось выяснить до сих пор, она смогла получить метаданные каждой сессии, которую хакеры использовали для подключения к их серверам. Вот особенности данных:
  * 'Session_Connection_Time': Сколько длилась сессия в минутах
  * 'Bytes Transferred': Количество МБ, переданных во время сессии
  * 'Kali_Trace_Used': Указывает, использовал ли хакер Kali Linux
  * 'Servers_Corrupted': Количество серверов, поврежденных во время атаки
  * 'Pages_Corrupted': Количество незаконно доступных страниц
  * 'Location': Место, откуда произошла атака (вероятно, бесполезно, потому что хакеры использовали VPN)
  * 'WPM_Typing_Speed': Их оценочная скорость печати на основе журналов сессий.

- У технологической компании есть 3 потенциальных хакера, которые совершили атаку. Они уверены в первых двух хакерах, но не очень уверены, был ли вовлечен третий хакер или нет.

- Вам необходимо выяснить, был ли третий подозреваемый как-то связан с атаками, или это были всего два хакера? Вероятно, наверняка это знать невозможно, но, возможно, то, что вы только что узнали о кластеризации, может помочь!

**Еще один ключевой факт, судебный инженер знает, что хакеры чередуют атаки. Это означает, что каждый из них должен иметь примерно одинаковое количество атак. Например, если было 100 общих атак, то в ситуации с двумя хакерами каждый должен был бы сделать около 50 взломов, в ситуации с тремя хакерами - каждый бы сделал около 33 взломов. Инженер считает, что это ключевой элемент для решения этой проблемы, но не знает, как различить эти неразмеченные данные на группы хакеров.**

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [ ]:
from pyspark.ml.clustering import KMeans

dataset = spark.read.csv("data/hack_data.csv",header=True,inferSchema=True)

In [ ]:
dataset.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [ ]:
dataset.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       NULL|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       NULL| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [ ]:
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [ ]:
dataset = dataset.drop('Location')

In [ ]:
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

In [ ]:
# Импортируем необходимые классы из модуля pyspark.ml.feature
from pyspark.ml.feature import (
    VectorAssembler,  # Класс для сбора нескольких столбцов в один векторный столбец
    VectorIndexer,    # Класс для автоматического индексирования категориальных признаков
    OneHotEncoder,    # Класс для преобразования категориальных признаков в бинарные векторы
    StringIndexer     # Класс для индексирования строковых столбцов
)

In [ ]:
# # Создаем объект StringIndexer. Этот объект преобразует строковые значения в индексы.
# # Входной столбец - 'Sex', выходной столбец - 'SexIndex'.
# location_indexer = StringIndexer(inputCol='Location',outputCol='LocationIndex', handleInvalid="keep")

# # Создаем объект OneHotEncoder. Этот объект преобразует категориальные признаки, закодированные как индексы, в векторы One-Hot.
# # Входной столбец - 'SexIndex', выходной столбец - 'SexVec'.
# location_encoder = OneHotEncoder(inputCol='LocationIndex',outputCol='LocationVec')

In [ ]:
feature_cols = ['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used', 'Servers_Corrupted', 'Pages_Corrupted', 'WPM_Typing_Speed']

assembler = VectorAssembler(inputCols=feature_cols, outputCol="nonsc_features")

In [ ]:
# Обучение модели k-means.
kmeans = KMeans(featuresCol='features', k=2).setSeed(1)

In [ ]:
# Импорт StandardScaler из библиотеки pyspark.ml

from pyspark.ml.feature import StandardScaler

In [ ]:
# Создание экземпляра StandardScaler

scaler = StandardScaler(inputCol="nonsc_features", outputCol="features", withStd=True, withMean=False)

In [ ]:
# Создаем объект Pipeline. Этот объект представляет собой последовательность этапов обработки данных.
# Этапы включают: gender_indexer, embark_indexer, gender_encoder, embark_encoder, assembler, log_reg_titanic.
pipeline = Pipeline(stages=[
    # location_indexer,  # Преобразование строковых значений в 'Sex' в индексы
    # location_encoder,# Преобразование строковых значений в 'Embarked' в индексы
    assembler,
    scaler,# Объединение нескольких признаков в один векторный столбец 'features'
    kmeans  # Применение модели логистической регрессии
])

In [ ]:
# Метод randomSplit разделяет исходные данные на два набора данных в заданной пропорции.
# В данном случае, мы разделяем данные на обучающий набор (70% данных) и тестовый набор (30% данных).

train, test = dataset.randomSplit([0.7,.3])

In [ ]:
# Метод fit обучает модель на обучающих данных.
# В данном случае, мы обучаем наш конвейер (pipeline) на обучающем наборе данных (train_titanic_data).

fit_model = pipeline.fit(dataset)

In [ ]:
# Метод transform применяет обученную модель к новым данным.
# В данном случае, мы применяем нашу обученную модель (fit_model) к тестовому набору данных (test_titanic_data).

results = fit_model.transform(dataset)

In [ ]:
results.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [ ]:
full_results = fit_model.transform(dataset)

In [ ]:
kmeans_model = fit_model.stages[-1]

In [ ]:
# Оценка кластеризации путем вычисления суммы квадратов ошибок внутри групп.
wssse = kmeans_model.summary.trainingCost
print("Сумма квадратов ошибок внутри групп = " + str(wssse))

# Вывод результатов.
centers = kmeans_model.clusterCenters()
print("Центры кластеров: ")
for center in centers:
    print(center)
print(f"Кластер 1: {centers[0]}")
print(f"Кластер 2: {centers[1]}")
print(f"Кластер 3: {centers[2]}")

Сумма квадратов ошибок внутри групп = 601.7707512676691
Центры кластеров: 
[2.99991988 2.92319035 1.05261534 3.20390443 4.51321315 3.28474   ]
[1.26023837 1.31829808 0.99280765 1.36491885 2.5625043  5.26676612]
Кластер 1: [2.99991988 2.92319035 1.05261534 3.20390443 4.51321315 3.28474   ]
Кластер 2: [1.26023837 1.31829808 0.99280765 1.36491885 2.5625043  5.26676612]


IndexError: list index out of range

# Задание 5. Создание рекомендательной системы

- Необходимо построить систему рекомендаций для компании по доставке еды онлайн.

- Эта компания пытается выделиться на рынке, рекомендуя новые блюда клиентам на основе предпочтений других клиентов.

- **Ваш окончательный результат должен быть в виде функции, которая может принимать DataFrame Spark отдельного клиента с оценками различных блюд и выводить три лучших предложенных блюда.**

In [ ]:
import pandas as pd

In [ ]:
spark = SparkSession.builder.appName("RecommendationSystem").getOrCreate()

In [ ]:
df = pd.read_csv('data/movielens_ratings.csv')

In [ ]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
movieId,1501.0,49.405730,28.937034,0.0,24.0,50.0,74.0,99.0
rating,1501.0,1.774151,1.187276,1.0,1.0,1.0,2.0,5.0
userId,1501.0,14.383744,8.591040,0.0,7.0,14.0,22.0,29.0


In [ ]:
df.corr()

,movieId,rating,userId
movieId,1.000000,0.036569,0.003267
rating,0.036569,1.000000,0.056411
userId,0.003267,0.056411,1.000000


In [ ]:
import numpy as np
df['mealskew'] = df['movieId'].apply(lambda id: np.nan if id > 31 else id)

In [ ]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
movieId,1501.0,49.405730,28.937034,0.0,24.0,50.0,74.0,99.0
rating,1501.0,1.774151,1.187276,1.0,1.0,1.0,2.0,5.0
userId,1501.0,14.383744,8.591040,0.0,7.0,14.0,22.0,29.0
mealskew,486.0,15.502058,9.250634,0.0,7.0,15.0,23.0,31.0


In [ ]:
mealmap = { 2. : "Chicken Curry",
           3. : "Spicy Chicken Nuggest",
           5. : "Hamburger",
           9. : "Taco Surprise",
           11. : "Meatloaf",
           12. : "Ceaser Salad",
           15. : "BBQ Ribs",
           17. : "Sushi Plate",
           19. : "Cheesesteak Sandwhich",
           21. : "Lasagna",
           23. : "Orange Chicken",
           26. : "Spicy Beef Plate",
           27. : "Salmon with Mashed Potatoes",
           28. : "Penne Tomatoe Pasta",
           29. : "Pork Sliders",
           30. : "Vietnamese Sandwich",
           31. : "Chicken Wrap",
           np.nan: "Cowboy Burger",
           4. : "Pretzels and Cheese Plate",
           6. : "Spicy Pork Sliders",
           13. : "Mandarin Chicken PLate",
           14. : "Kung Pao Chicken",
           16. : "Fried Rice Plate",
           8. : "Chicken Chow Mein",
           10. : "Roasted Eggplant ",
           18. : "Pepperoni Pizza",
           22. : "Pulled Pork Plate",
           0. : "Cheese Pizza",
           1. : "Burrito",
           7. : "Nachos",
           24. : "Chili",
           20. : "Southwest Salad",
           25.: "Roast Beef Sandwich"}

In [ ]:
df['meal_name'] = df['mealskew'].map(mealmap)

In [ ]:
df.to_csv('data/Meal_Info.csv',index=False)

In [ ]:
data = spark.read.csv('data/Meal_Info.csv',inferSchema=True,header=True)

In [ ]:
data.show()

+-------+------+------+--------+--------------------+
|movieId|rating|userId|mealskew|           meal_name|
+-------+------+------+--------+--------------------+
|      2|   3.0|     0|     2.0|       Chicken Curry|
|      3|   1.0|     0|     3.0|Spicy Chicken Nug...|
|      5|   2.0|     0|     5.0|           Hamburger|
|      9|   4.0|     0|     9.0|       Taco Surprise|
|     11|   1.0|     0|    11.0|            Meatloaf|
|     12|   2.0|     0|    12.0|        Ceaser Salad|
|     15|   1.0|     0|    15.0|            BBQ Ribs|
|     17|   1.0|     0|    17.0|         Sushi Plate|
|     19|   1.0|     0|    19.0|Cheesesteak Sandw...|
|     21|   1.0|     0|    21.0|             Lasagna|
|     23|   1.0|     0|    23.0|      Orange Chicken|
|     26|   3.0|     0|    26.0|    Spicy Beef Plate|
|     27|   1.0|     0|    27.0|Salmon with Mashe...|
|     28|   1.0|     0|    28.0| Penne Tomatoe Pasta|
|     29|   1.0|     0|    29.0|        Pork Sliders|
|     30|   1.0|     0|    3

________

In [ ]:
(training, test) = data.randomSplit([0.8, 0.2])

In [ ]:
from pyspark.ml.recommendation import ALS

In [ ]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")

In [ ]:
model = als.fit(training)

In [ ]:
predictions = model.transform(test)

In [ ]:
predictions.show()

+-------+------+------+--------+--------------------+-----------+
|movieId|rating|userId|mealskew|           meal_name| prediction|
+-------+------+------+--------+--------------------+-----------+
|      1|   1.0|    28|     1.0|             Burrito|   3.990017|
|      7|   1.0|    28|     7.0|              Nachos|  0.8110689|
|      4|   4.0|    26|     4.0|Pretzels and Chee...|  2.8526647|
|      7|   5.0|    26|     7.0|              Nachos|  3.7708843|
|      7|   3.0|    12|     7.0|              Nachos|  -2.241238|
|      8|   1.0|    12|     8.0|   Chicken Chow Mein| -2.4856234|
|      3|   2.0|    22|     3.0|Spicy Chicken Nug...| -3.5740156|
|      6|   2.0|    22|     6.0|  Spicy Pork Sliders|  1.5875158|
|      9|   1.0|    22|     9.0|       Taco Surprise|    -3.1897|
|      4|   2.0|     1|     4.0|Pretzels and Chee...|  0.6281193|
|      6|   1.0|    13|     6.0|  Spicy Pork Sliders| 0.93239814|
|      0|   1.0|     5|     0.0|        Cheese Pizza|  1.7875237|
|      0| 

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")

In [ ]:
rmse = evaluator.evaluate(predictions)

print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.8510391195156752


In [ ]:
def recommend_for_user(user_df, num_recommend=3):
    recommend = model.transform(user_df)
    recommend = recommend.orderBy('prediction', ascending=False)
    top_recommend = recommend.limit(num_recommend)
    top_recommend = top_recommend.join(data, on='movieId', how='inner') \
                                 .select('movieId', 'meal_name', 'prediction') \
                                 .distinct()

    return top_recommend

In [ ]:
single_user = test.filter(test['userId'] == 11).select(['movieId', 'userId'])
recommend = recommend_for_user(single_user)
recommend.show()

+-------+-------------------+----------+
|movieId|          meal_name|prediction|
+-------+-------------------+----------+
|     30|Vietnamese Sandwich| 1.6688204|
|     18|    Pepperoni Pizza| 4.5914364|
|     70|      Cowboy Burger| 3.4592037|
+-------+-------------------+----------+

